In [63]:
# Para utilizar ferramentas matemáticas
import numpy as np

# Para utilizar simulador e hardware quântico real
from qiskit import Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.providers.aer import QasmSimulator
from qiskit_textbook.tools import simon_oracle
# Para construir e manipular circuitos
from qiskit import QuantumCircuit, assemble, transpile

# Para utilizar ferramentas de visualização
from qiskit.visualization import plot_histogram

# Oráculo de Simon
def simon_oracle2(b):
    n = len(b)
    oracle_qc = QuantumCircuit(2 * n,n)
    
    # Copia o 1o registrador no 2o registrador
    for i in range(n):
        oracle_qc.cx(i, i + n)
    
    # Encontra o índice menos significativo e inclui as portas CNOT necessárias
    for i in range (n):
        if (b[i] == '1'):
            for j in range (i, n):
                if (b[j] == '1'):
                    oracle_qc.cx(i, j+n)
            break
    return oracle_qc

# String secreta
b = '110'

# Faz b seguir a ordenação de bits do Qiskit
b = b[::-1] 

# Número de qubits dos registradores
n = len(b)

# Cria o circuito quântico
simon_circuit = QuantumCircuit(n * 2,n)

# Aplica Hadamard ao 1o registrador
simon_circuit.h(range(n))

# Inclui uma barra para facilitar a visualização do circuito
simon_circuit.barrier()

# Inclui o oráculo de Simon
simon_circuit += simon_oracle(b)

# Inclui uma barra para facilitar a visualização do circuito
simon_circuit.barrier()

# Aplica Hadamard novamente ao 1o registrador
simon_circuit.h(range(n))

# Inclui uma barra para facilitar a visualização do circuito
simon_circuit.barrier()

# Realiza a medição
simon_circuit.measure(range(n), range(n))
simon_circuit.draw()

TypeError: unsupported operand type(s) for +=: 'QuantumCircuit' and 'QuantumCircuit'

In [57]:
# Excecuta em um simulador
aer_sim = Aer.get_backend('aer_simulator')
qobj = assemble(simon_circuit)

# Obtém os resultados
results = aer_sim.run(qobj).result()
counts = results.get_counts()

# Mostra os resultados
plot_histogram(counts)

/var/folders/30/pgp77_r92w5_rn1ghx4kn51h0000gn/T/ipykernel_26186/678314940.py:6: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  results = aer_sim.run(qobj).result()


QiskitError: 'No counts for experiment "0"'

In [ ]:
# String secreta
b = '10'

n = len(b)
simon_circuit = QuantumCircuit(n*2, n)

# Aplica Hadamard ao 1o registrador
simon_circuit.h(range(n))

# Aplica uma barreira para facilitar a visualização
simon_circuit.barrier()

# Inclui o oráculo de Simon
simon_circuit += simon_oracle(b)

# Aplica uma barreira para facilitar a visualização
simon_circuit.barrier()

# Aplica Hadamard ao 2o registrador
simon_circuit.h(range(n))

# Aplica uma barreira para facilitar a visualização
simon_circuit.barrier()

# Realiza a medição
simon_circuit.measure(range(n), range(n))
simon_circuit.draw()

TypeError: unsupported operand type(s) for +=: 'QuantumCircuit' and 'QuantumCircuit'

In [ ]:
# Carrega uma conta do IBMQ e obtém o dispositivo menos ocupado que tenha um número de qubits maior ou igual a (n+1)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2*n and
                                   not x.configuration().simulator and x.status().operational==True))

# Transpila e executa
shots = 1024
transpiled_bv_circuit = transpile(simon_circuit, backend)
job = backend.run(transpiled_bv_circuit, shots=shots)

# Obtém os resultados
results = job.result()
counts = results.get_counts()

# Mostra os resultados
plot_histogram(counts)

IBMQAccountCredentialsNotFound: 'No IBM Quantum Experience credentials found.'